# SRT Lab: Galactic Rotation Curve

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import os
from astropy.io import fits
from astrocalc import *

In [ ]:
#Specify directory containing fits images for each galactic longitude:
datadir = 'srt-galactic-test'
imgs = [fits.open(os.path.join(datadir,f)) for f in os.listdir(datadir) if f.endswith('.fits')]
objnames = [f.split('.')[0] for f in os.listdir(datadir) if f.endswith('.fits')]
vals = []
freq = []
for hdul in imgs:
    data = hdul[0].data/len(hdul)
    f_s = 2e6
    n_spec = len(data)
    for i in range(1,len(hdul)):
        data = data + hdul[i].data/len(hdul)
    freqs = np.fft.fftfreq(n_spec,1./f_s)
    freqs = np.fft.fftshift(freqs)*1e-6+1420.4
    vals.append(data)
    freq.append(freqs)
srtdata = pd.DataFrame({'value':vals,'frequency':freq,'objs':objnames})
srtdata = srtdata.set_index('objs')
print(srtdata.loc['g10'])

## Plot Integrated Power Spectrum

In [ ]:
#Display integrated power spectrum for given galactic longitude
obj = 'g40' #set obj to desired fits file to display (no .fits extension needed)
fig = px.scatter(x=srtdata['frequency'].loc[obj],y=srtdata['value'].loc[obj])
fig.update_layout(
    title="Integrated Power Spectrum for "+ obj, xaxis_title="Frequency (MHz)", yaxis_title="Value",
)
fig.show()

## Calculate V_lsr and Relative Velocity

In [ ]:
#calculate v_lsr
v_lsrs = []
for hdul in imgs:
    lat = 42.5 #at Haystack
    lon = 71.5 #at Haystack
    date = hdul[0].header['Date-OBS']
    year= int(hdul[0].header['Date-OBS'].split('-')[0])
    day = dayOfYear(date)
    time = hdul[0].header['UTC']
    hour,minute,sec = map(float,time.split(":"))
    metadata = hdul[0].header['METADATA'].split(',')
    for element in metadata:
        if "motor_az" in element:
            az = float(element.split(':')[-1])
        elif "motor_el" in element:
            el = float(element.split(':')[-1])
    v_lsr = calc_vlsr(year,day,hour,minute,sec,az,el,lat,lon)
    v_lsrs.append(v_lsr)
srtdata['v_lsr'] = v_lsrs

#use doppler shift to calculate relative velocity for each frequency bin
lightspeed = 299792
center_freq = 1420.4
velocities = []
for i in range(len(srtdata)):
    velocities.append([((center_freq-f)/center_freq)*lightspeed - srtdata['v_lsr'].iloc[i] for f in srtdata['frequency'].iloc[i]])
srtdata['velocity'] = velocities

## Plot Corrected Power vs Velocity

In [ ]:
obj = 'g40' #set obj to desired fits file to display (no .fits extension needed)
data = {'velocity':srtdata['velocity'].loc[obj],'frequency':srtdata['frequency'].loc[obj],'value':srtdata['value'].loc[obj]}
fig = px.scatter(data, x="velocity",y="value",hover_data=["frequency"])
fig.update_layout(
    title="Corrected Power Spectrum for " + obj, xaxis_title="Velocity (Km/s)", yaxis_title="Value",
)
fig.show()

## Plot Galactic Rotation Curve

In [ ]:
#enter list of observed galactic longitudes and max-observed velocities that you observed using the power vs velocity graphs above
#list length should equal the number of galactic longitudes observed
l = [0,10,20,30,40,50,60,70,80,90]
vmax_obs = [61,75,120,121,97,92,60,40,37,28] #user enter

#calculated values based on galactic longitude
r= [0,1.5,2.9,4.2,5.5,6.5,7.4,8.0,8.37,8.5] #kpc #R = R_0*sin(l)
w0r = [0,39,75,109,142,168,192,207,217,220]

#calculate rotational velocity using observations
v = [vmax_obs[i] + w0r[i] for i in range(len(vmax_obs))]
print(v)

#calculate 
fig = go.Figure(data=go.Scatter(x=r[2:], y=v[2:], mode="markers"))
fig.update_layout(
    title="Galactic Rotation Curve", xaxis_title="Distance from Galactic Center R (kpc)", 
    yaxis_title="Galactic Rotation Velocity (km/s)")
fig.update_yaxes(range=[0,300])
fig.update_xaxes(range=[0,10])
fig.show()